# Split Stages

It is also possible to split stages.
We start of with the stage $k$. We want to split this stage up into the stages $\alpha$ and $\beta$.
If both stages are combined back we want to obtian the original stage. Therefore we know that

$$A_k=A_\beta A_\alpha$$

$$B_k=\begin{bmatrix}A_\beta B_\alpha & B_\beta \end{bmatrix}$$

$$C_k=\begin{bmatrix}C_\alpha \\ C_\beta A_\alpha \end{bmatrix}$$

$$D_k = \begin{bmatrix} D_\alpha & 0 \\ C_\beta A_\alpha & D_\beta \end{bmatrix}$$



Now we want to compute these matices.
For this we consider the Hankel matrix associated with the new intermediate step.

This is 
$$H = \mathcal{O}_\diamond \mathcal{R}_\diamond 
= \begin{bmatrix} C_\beta \\ C_{k+1} A_\beta \\ C_{k+2} A_{k+1} A_\beta \\ \vdots \end{bmatrix}
\begin{bmatrix} \cdots& A_\alpha A_{k-1}B_{k-2} & A_\alpha B_{k-1}&  B_\alpha  \end{bmatrix}
$$

By factoring $ \mathcal{O}_\diamond $ and $\mathcal{R}_\diamond $ we obtain.
$$H
= \begin{bmatrix} 1 &\\ & \mathcal{O}_{k+1} \end{bmatrix} \begin{bmatrix} C_\beta \\  A_\beta \end{bmatrix}
\begin{bmatrix} A_\alpha &  B_\alpha  \end{bmatrix} \begin{bmatrix} \mathcal{R}_k\\&1  \end{bmatrix}$$

and by multipling the inner of the three products we get the matrix 

$$\begin{bmatrix} C_\beta \\  A_\beta \end{bmatrix}
\begin{bmatrix} A_\alpha &  B_\alpha  \end{bmatrix}
=
\begin{bmatrix}
C_\beta A_\alpha & C_\beta B_\alpha\\
A_\beta A_\alpha & A_\beta B_\alpha
\end{bmatrix}
$$

With the previous equations we can identify this matrix as 

$$
\begin{bmatrix}
C_\beta A_\alpha & C_\beta B_\alpha\\
A_\beta A_\alpha & A_\beta B_\alpha
\end{bmatrix}
=
\begin{bmatrix}
C_{k[2]} & D_{k[2,1]}\\
A_k & B_{k[1]}
\end{bmatrix}
$$

By computing the SVD we can now get the matricies
$$
\begin{bmatrix}
C_k & D_{k[1,0]}\\
A_k & B_k
\end{bmatrix} = U \Sigma V^\top = \begin{bmatrix} C_\beta \\  A_\beta \end{bmatrix}
\begin{bmatrix} A_\alpha &  B_\alpha  \end{bmatrix}
$$

If $O_{k+1}^\top O_{k+1} = 1$ and $R_k  R_k^\top = 1$ the singluar values calcualted by the svd are also the Hankel singualr values.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tvsclib.stage import Stage
from tvsclib.strict_system import StrictSystem
from tvsclib.mixed_system import MixedSystem
import tvsclib.utils as utils
import tvsclib.math as math

In [ ]:
def split_stage(stage,i_in,i_out):
    A=stage.A_matrix
    B=stage.B_matrix
    C=stage.C_matrix
    D=stage.D_matrix
    (d_stateo,d_statei)=A.shape
    U,s,Vt = np.linalg.svd(np.block([[C[i_out:,:],D[i_out:,:i_in]],[A,B[:,:i_in]]]),full_matrices=False)
    
    s_sqrt = np.sqrt(s)
    stage_alpha=Stage(s_sqrt.reshape(-1,1)*Vt[:,:d_statei],\
                 s_sqrt.reshape(-1,1)*Vt[:,d_statei:],\
                 C[:i_out,:],\
                 D[:i_out,:i_in])
    stage_beta=Stage(U[U.shape[0]-d_stateo:,:]*s_sqrt.reshape(1,-1),\
               B[:,i_in:],\
               U[:U.shape[0]-d_stateo,:]*s_sqrt.reshape(1,-1),\
               D[i_out:,i_in:])
    
    return stage_alpha,stage_beta
    
def split_stage_anti(stage,i_in,i_out,D):
    A=stage.A_matrix
    B=stage.B_matrix
    C=stage.C_matrix
    #D=stage.D_matrix
    (d_stateo,d_statei)=A.shape
    U,s,Vt = np.linalg.svd(np.block([[C[:i_out,:],D[:i_out,i_in:]],[A,B[:,i_in:]]]),full_matrices=False)
    
    s_sqrt = np.sqrt(s)
    stage_alpha=Stage(s_sqrt.reshape(-1,1)*Vt[:,:d_statei],\
                 s_sqrt.reshape(-1,1)*Vt[:,d_statei:],\
                 C[i_out:,:],\
                 np.zeros_like(D[:i_out,:i_in]))
    stage_beta=Stage(U[U.shape[0]-d_stateo:,:]*s_sqrt.reshape(1,-1),\
               B[:,:i_in],\
               U[:U.shape[0]-d_stateo,:]*s_sqrt.reshape(1,-1),\
               np.zeros_like(D[i_out:,i_in:]))
    
    return stage_alpha,stage_beta
    
def split(system,k,i_in,i_out):
    stage_alpha,stage_beta = split_stage(system.stages[k],i_in,i_out)
    system.stages[k]=stage_alpha
    system.stages.insert(k+1,stage_beta)
    
def split_anti(system,k,i_in,i_out,D):
    stage_alpha,stage_beta = split_stage_anti(system.stages[k],i_in,i_out,D)
    system.stages[k]=stage_beta
    system.stages.insert(k+1,stage_alpha)
    
def split_mixed(system,k,i_in,i_out):
    split_anti(system.anticausal_system,k,i_in,i_out,system.causal_system.stages[k].D_matrix)
    split(system.causal_system,k,i_in,i_out)
    

In [ ]:
def initial(T):
    """
    
    returns a tupel with the initial lists
    for As,Bs,Cs,Ds
    
    """
    
    return StrictSystem(stages=[Stage(np.zeros((0,0)),np.zeros((0,T.shape[1])),np.zeros((T.shape[0],0)),T)],causal=True)

def initial_mixed(T):
    """
    
    returns a tupel with the initial lists
    for As,Bs,Cs,Ds
    
    """
    
    return MixedSystem(\
            causal_system=StrictSystem(stages=[Stage(np.zeros((0,0)),np.zeros((0,T.shape[1])),np.zeros((T.shape[0],0)),T)],causal=True),\
        anticausal_system=StrictSystem(stages=[Stage(np.zeros((0,0)),np.zeros((0,T.shape[1])),np.zeros((T.shape[0],0)),np.zeros_like(T))],causal=False)\
                      )

In [ ]:
T = np.random.rand(16,16)

In [ ]:
system = initial_mixed(T)
utils.show_system(system,mark_D=False)
utils.check_dims(system)

In [ ]:
system.anticausal_system.stages[0].A_matrix

In [ ]:
split_mixed(system,0,8,8)
utils.show_system(system,mark_D=False)

In [ ]:
system.anticausal_system.stages[1].B_matrix.shape

In [ ]:
split_mixed(system,0,4,4)
utils.show_system(system,mark_D=False)

In [ ]:
split_mixed(system,2,4,4)
utils.show_system(system,mark_D=False)

In [ ]:
np.max(np.abs(T-system.to_matrix()))

In [ ]:
class A_matrix:
    def __init__(self,s_out,A,s_in):
        self.A = A
        self.s_out = s_out
        self.s_in = s_in
    
    @property
    def input_normal(self):
        return self.A*self.s_in.reshape(1,-1)

    @property
    def output_normal(self):
        return self.s_out.reshape(-1,1)*self.A

    @property
    def balanced(self):
        return np.sqrt(self.s_out.reshape(-1,1))*self.A*np.sqrt(self.s_in.reshape(1,-1))
 
    @property
    def output_input_normal(self):
        """
        Thsi can be used to get a stage where the previous R_k is input normal 
        and the next O_{k+1} is output normal
        """
        return self.s_out.reshape(-1,1)*self.A*self.s_in.reshape(1,-1)
    
    def copy(self):
        return (A_matrix(self.s_out.copy(),self.A.copy(),self.s_in.copy()))
    
class B_matrix:
    def __init__(self,s_out,B):
        self.B = B
        self.s_out = s_out
    
    @property
    def input_normal(self):
        return self.B

    @property
    def output_normal(self):
        return self.s_out.reshape(-1,1)*self.B

    @property
    def blanced(self):
        return np.sqrt(self.s_out.reshape(-1,1))*self.B

    def copy(self):
        return (B_matrix(self.s_out.copy(),self.B.copy()))
    
class C_matrix:
    def __init__(self,C,s_in):
        self.C = C
        self.s_in = s_in
    
    @property
    def input_normal(self):
        return self.C*self.s_in.reshape(1,-1)

    @property
    def output_normal(self):
        return self.C

    @property
    def balanced(self):
        return self.C*np.sqrt(self.s_in.reshape(1,-1))
    
    def copy(self):
        return (C_matrix(self.C.copy(),self.s_in.copy()))

In [ ]:
def split_stage_sigmas(stage,i_in,i_out):
    #we need thsi stage such that previous is input normal and later is output normal
    s_in = stage.A_matrix.s_in
    s_out= stage.A_matrix.s_out
    A=stage.A_matrix.output_input_normal
    B=stage.B_matrix.output_normal
    C=stage.C_matrix.input_normal
    D=stage.D_matrix

    (d_stateo,d_statei)=A.shape
    U,s,Vt = np.linalg.svd(np.block([[C[i_out:,:],D[i_out:,:i_in]],[A,B[:,:i_in]]]),full_matrices=False)
    

    stage_alpha=Stage(
                 A_matrix(s,Vt[:,:d_statei]/s_in.reshape(1,-1),s_in),\
                 B_matrix(s,Vt[:,d_statei:]),\
                 C_matrix(stage.C_matrix.C[:i_out,:],s_in),\
                 D[:i_out,:i_in])
    stage_beta=Stage(
               A_matrix(s_out,U[U.shape[0]-d_stateo:,:]/s_out.reshape(-1,1),s),\
               B_matrix(s_out,stage.B_matrix.B[:,i_in:]),\
               C_matrix(U[:U.shape[0]-d_stateo,:],s),\
               D[i_out:,i_in:])
    
    return stage_alpha,stage_beta

def split_stage_sigmas_anti(stage,i_in,i_out,D):
    #we need thsi stage such that previous is input normal and later is output normal
    s_in = stage.A_matrix.s_in
    s_out= stage.A_matrix.s_out
    A=stage.A_matrix.output_input_normal
    B=stage.B_matrix.output_normal
    C=stage.C_matrix.input_normal
    #D=stage.D_matrix

    (d_stateo,d_statei)=A.shape
    U,s,Vt = np.linalg.svd(np.block([[C[:i_out,:],D[:i_out,i_in:]],[A,B[:,i_in:]]]),full_matrices=False)
    

    stage_alpha=Stage(
                 A_matrix(s,Vt[:,:d_statei]/s_in.reshape(1,-1),s_in),\
                 B_matrix(s,Vt[:,d_statei:]),\
                 C_matrix(stage.C_matrix.C[i_out:,:],s_in),\
                 np.zeros_like(D[:i_out,:i_in]))
    stage_beta=Stage(
               A_matrix(s_out,U[U.shape[0]-d_stateo:,:]/s_out.reshape(-1,1),s),\
               B_matrix(s_out,stage.B_matrix.B[:,:i_in]),\
               C_matrix(U[:U.shape[0]-d_stateo,:],s),\
               np.zeros_like(D[i_out:,i_in:]))
    return stage_alpha,stage_beta

def split_sigmas(system,k,i_in,i_out):
    stage_alpha,stage_beta = split_stage_sigmas(system.stages[k],i_in,i_out)
    system.stages[k]=stage_alpha
    system.stages.insert(k+1,stage_beta)
    
def split_sigmas_anti(system,k,i_in,i_out,D):
    stage_alpha,stage_beta = split_stage_sigmas_anti(system.stages[k],i_in,i_out,D)
    system.stages[k]=stage_beta
    system.stages.insert(k+1,stage_alpha)
    
def split_sigmas_mixed(system,k,i_in,i_out):
    split_sigmas_anti(system.anticausal_system,k,i_in,i_out,system.causal_system.stages[k].D_matrix)
    split_sigmas(system.causal_system,k,i_in,i_out)

In [ ]:
def initial_sigmas(T):
    """
    
    returns a tupel with the initial lists
    for As,Bs,Cs,Ds
    
    """
    
    return StrictSystem(stages=[Stage(A_matrix(np.zeros(0),np.zeros((0,0)),np.zeros(0)),\
                                      B_matrix(np.zeros(0),np.zeros((0,T.shape[1]))),\
                                      C_matrix(np.zeros((T.shape[0],0)),np.zeros(0)),\
                                      T)],causal=True)

def initial_sigmas_mixed(T):
    """
    
    returns a tupel with the initial lists
    for As,Bs,Cs,Ds
    
    """
    return MixedSystem(\
                causal_system=StrictSystem(stages=[Stage(A_matrix(np.zeros(0),np.zeros((0,0)),np.zeros(0)),\
                                      B_matrix(np.zeros(0),np.zeros((0,T.shape[1]))),\
                                      C_matrix(np.zeros((T.shape[0],0)),np.zeros(0)),\
                                      T)],causal=True),
                anticausal_system=StrictSystem(stages=[Stage(A_matrix(np.zeros(0),np.zeros((0,0)),np.zeros(0)),\
                                      B_matrix(np.zeros(0),np.zeros((0,T.shape[1]))),\
                                      C_matrix(np.zeros((T.shape[0],0)),np.zeros(0)),\
                                      np.zeros_like(T))],causal=False))

def get_system(system):
    stages = []
    for stage in system.stages:
        stages.append(Stage(stage.A_matrix.input_normal,\
                            stage.B_matrix.input_normal,\
                            stage.C_matrix.input_normal,\
                            stage.D_matrix))
    return StrictSystem(stages=stages,causal=system.causal)

def get_system_mixed(system):
    
    return MixedSystem(causal_system=get_system(system.causal_system),anticausal_system=get_system(system.anticausal_system))

In [ ]:
sys = initial_sigmas_mixed(T)
utils.show_system(get_system_mixed(sys),mark_D=False)

In [ ]:
split_sigmas_mixed(sys,0,8,8)
utils.show_system(get_system_mixed(sys),mark_D=False)

In [ ]:
split_sigmas_mixed(sys,0,4,4)
utils.show_system(get_system_mixed(sys),mark_D=False)

In [ ]:
split_sigmas_mixed(sys,2,4,4)
utils.show_system(get_system_mixed(sys),mark_D=False)

In [ ]:
split_sigmas_mixed(sys,1,2,2)
split_sigmas_mixed(sys,3,2,2)
utils.show_system(get_system_mixed(sys),mark_D=False)

In [ ]:
#np.max(np.abs(np.tril(get_system(sys).to_matrix())-np.tril(T)))
np.max(np.abs(get_system_mixed(sys).to_matrix()-T))


In [ ]:
math.extract_sigmas(T,get_system_mixed(sys).dims_in,get_system_mixed(sys).dims_out)

In [ ]:
[stage.A_matrix.s_in for stage in sys.causal_system.stages][1:]

In [ ]:
[stage.A_matrix.s_in for stage in sys.anticausal_system.stages][:-1]